In [1]:
from QuantumChessGame import *
from MinimalQCGameState import *

# Use the compiled AI, not the python version
from QuantumChessAI import *
#from ExpectiMinimaxAI import *

from MCTS import MCTS_Node

from ChessPuzzles import chess_puzzles
import numpy as np
import time
import random

### Minimax vs MCTS

In [3]:
class MCTS_AI:
    def __init__(self):
        return
    
    def find_best_move(self, game, UseMinimalGameState=True):
        
        
        if UseMinimalGameState:
            # if using the minimal game state, derive game state from the game to build MCTS root 
            init_state = game.config['initial_state_fen']
            move_history = game.get_history()
            move_code = game.movecode.value
            gamestate = MinimalQuantumChessGameState(init_state, move_history, move_code) 
            root = MCTS_Node(gamestate)   
        else:
            #if using full QC game structure
            root = MCTS_Node(game)
        
        gamedata = game.get_game_data()
        bestmove = root.best_action(gamedata.ply)
        return bestmove

In [4]:
# Set up a new game
game = QuantumChessGame()
game.new_game({})

# Create Minimax AI
AIConfig1 = AISearchConfig()
AIConfig1.max_search_depth = 5
AIConfig1.max_search_time = 4000
AIConfig1.capture_move_repetitions = 1
AIConfig1.use_transposition_table = False
AIConfig1.use_iterative_deepening = False
AIConfig1.search_quantum_moves = False
AI = QuantumChessAI()
AI.set_configuration(AIConfig1)

# Create MCTS AI
MCTSAI = MCTS_AI()

movecode = 1
gamedata = game.get_game_data()
move_history = []
random.seed(0)
while( movecode == 1 ):
    
    if( gamedata.ply % 2 == 0 ):
        searchresult = AI.find_best_move(game)
        move = str(searchresult.bestMove)[2:-1]
    else:
        move = MCTSAI.find_best_move(game)
    
    gamedata, movecode = game.do_move(move)
    move_history = game.get_history()
    print(move_history)
    
    if( len(move_history) > 200 ):
        break
    
print("Game ended with movecode: " + MoveCode[movecode])
AI.destroy()

AttributeError: function 'new_AI' not found

## Minimax classical vs Minimax quantum

In [ ]:
game = QuantumChessGame()
game.new_game()

AIConfig1 = AISearchConfig()
AIConfig1.max_search_depth = 3
AIConfig1.max_search_time = 2000
AIConfig1.capture_move_repetitions = 1
AIConfig1.use_transposition_table = False
AIConfig1.use_iterative_deepening = False
AIConfig1.search_quantum_moves = False

AIConfig2 = AISearchConfig()
AIConfig2.max_search_depth = 3
AIConfig2.max_search_time = 2000
AIConfig2.capture_move_repetitions = 25
AIConfig2.use_transposition_table = False
AIConfig2.use_iterative_deepening = False
AIConfig2.search_quantum_moves = True

AIs = [QuantumChessAI(),QuantumChessAI()]
AIs[0].set_configuration(AIConfig1)
AIs[1].set_configuration(AIConfig2)

movecode = 1
gamedata = game.get_game_data()
move_history = []
random.seed(0)
while( movecode == 1 ):
    
    searchresult = AIs[gamedata.ply%2].find_best_move(game)
    move = str(searchresult.bestMove)[2:-1]
    
    # If no valid move found, just pick first legal one
    if( move == "" ):
        all_moves = format_moves(game.get_legal_moves())
        move = all_moves[random.randrange(0, len(all_moves))]
        print("No move returned, legal move picked is ")
        print(move)
    
    print("Move: %s"%move)
    gamedata, movecode = game.do_move(move)
    
print("Game ended with movecode: " + MoveCode[movecode])
AIs[0].destroy()
AIs[1].destroy()

#### Convert move history to string format for Unity Sandbox mode

In [ ]:
p = ""
for move in move_history:
    p += 'p' + move + " "
    
print(p)